In [1]:
import numpy as np
from instrumental import Q_, u
from instrumental import instrument, list_instruments
from instrumental.drivers.daq.ni import Task

import sys
import clr
import os

sys.path.append(r"C:\Program Files\Thorlabs\Kinesis")
os.getcwd()
clr.AddReference("Thorlabs.MotionControl.DeviceManagerCLI")
clr.AddReference("Thorlabs.MotionControl.FilterFlipperCLI")
from instrumental.drivers.motion.filter_flipper import Position
# from instrumental.drivers.motion.BPC203 import BPC203

# import matplotlib.pyplot as plt

C:\Users\euyehara\miniconda3\envs\instrumental\Lib\site-packages\cffi\cparser.py:436: UserWarning: #pragma in cdef() are entirely ignored. They should be removed for now, otherwise your code might behave differently in a future version of CFFI if #pragma support gets added. Note that '#pragma pack' needs to be replaced with the 'packed' keyword argument to cdef().
  warnings.warn(


In [2]:
paramset = list_instruments()
paramset


C:\Users\euyehara\miniconda3\envs\instrumental\Lib\site-packages\instrumental_lib-0.8.dev0-py3.12.egg\instrumental\drivers\motion\ecc100.py:24: UserWarning: Driver 'instrumental.drivers.motion.ecc100' is out of date and incompatible with the current Instrumental core. Pull requests fixing this are welcome.
  warnings.warn(
C:\Users\euyehara\miniconda3\envs\instrumental\Lib\site-packages\cffi\cparser.py:436: UserWarning: #pragma in cdef() are entirely ignored. They should be removed for now, otherwise your code might behave differently in a future version of CFFI if #pragma support gets added. Note that '#pragma pack' needs to be replaced with the 'packed' keyword argument to cdef().
  warnings.warn(


[<ParamSet[HPOSA] visa_address='GPIB0::23::INSTR'>,
 <ParamSet[UC480_Camera] serial=b'4103485838' model=b'C1285R12M' id=1>,
 <ParamSet[NIDAQ] name='Dev1' serial=20316885 model=b'USB-6259'>,
 <ParamSet[BPC203] serial='71829423'>,
 <ParamSet[Filter_Flipper] serial='37006093'>]

In [10]:
paramsets[0]

<ParamSet[Keithley2635A] visa_address='GPIB0::29::INSTR' manufacturer='Keithley Instruments Inc.' serial=' 1284375' model=' Model 2635A' version=' 2.1.6'>

In [11]:
# cam = instrument(paramset[0]) 
# cam.save_instrument('Thorlabs_camera')

# daq = instrument(paramset[1])
# daq.save_instrument('NIDAQ_USB-6259')

# stage = instrument(paramset[3])
# stage.save_instrument('NanoMax_stage')

# ff = instrument(paramset[3])
# ff.save_instrument('Thorlabs_FilterFlipper')

sm = instrument(paramsets[0])


ModuleNotFoundError: No module named 'laserdiodecontrollers'

In [27]:
stage = instrument("NanoMax_stage", reopen_policy='reuse')

In [28]:
stage.connect()

Connecting to BPC203:
	- connecting to controller 71829423 --> done
	- connecting channel 1 (x axis) --> done
	- connecting channel 2 (y axis) --> done
	- connecting channel 3 (z axis) --> done




In [2]:
from instrumental.conf import user_conf_dir
user_conf_dir

'C:\\Users\\elise\\AppData\\Local\\mabuchilab\\instrumental'

stage.save_instrument('NanoMax_stage')
from instrumental.drivers.motion.BPC203 import BPC203_stage

In [3]:
%cd ..
from experiment_control.srs_microscope.srs_microscope import *

C:\Program Files (x86)\Thorlabs


ModuleNotFoundError: No module named 'experiment_control'

In [3]:
import pyvisa
rm = pyvisa.ResourceManager()
raw_list = rm.list_resources("GPIB?*")
print(raw_list)

('GPIB0::INTFC', 'GPIB0::2::INSTR', 'GPIB0::5::INSTR', 'GPIB0::15::INSTR', 'GPIB0::18::INSTR', 'GPIB0::20::INSTR', 'GPIB0::23::INSTR', 'GPIB0::29::INSTR')


In [6]:
inst = rm.open_resource('GPIB0::29::INSTR')
idn = inst.query('*IDN?')
print(idn)


Keithley Instruments Inc., Model 2635A, 1284375, 2.1.6



ImportError: attempted relative import with no known parent package

In [3]:
from instrumental.drivers.laserdiodecontrollers.Keithley2635A import Keithley2635A
from instrumental.drivers import VisaMixin
from instrumental.drivers import ParamSet
# from instrumental.drivers import powersupplies
# sm = Keithley2635A(visa_address='GPIB0::29::INSTR')

In [6]:
import re
from enum import Enum

import pyvisa

# from ... import Q_, u
# from .. import ParamSet, SCPI_Facet, VisaMixin
# from . import PowerSupply

# _INST_PARAMS_ = ['visa_address']
# _INST_VISA_INFO_ = {
#     'DP700': ('RIGOL TECHNOLOGIES', ['DP711', 'DP712']),
# }

def list_instruments():
    """Get a list of all power supplies currently attached"""
    paramsets = []
    search_string = "GPIB?*"
    rm = pyvisa.ResourceManager()
    raw_spec_list = rm.list_resources(search_string)

    for spec in raw_spec_list:
        try:
            inst = rm.open_resource(spec, read_termination='\n', write_termination='\n')
            idn = inst.query("*IDN?")
            manufacturer, model, serial, version = idn.rstrip().split(',', 4)
            if re.match(' Model 2635A', model):
                paramsets.append(ParamSet(Keithley2635A, visa_address=spec, manufacturer=manufacturer, serial=serial, model=model, version=version))
        except pyvisa.errors.VisaIOError as vio:
            # Ignore unknown serial devices
            pass

    return paramsets

In [8]:
paramsets = list_instruments()

In [9]:
paramsets

[<ParamSet[Keithley2635A] visa_address='GPIB0::29::INSTR' manufacturer='Keithley Instruments Inc.' serial=' 1284375' model=' Model 2635A' version=' 2.1.6'>]

In [4]:
import sys
import clr
import os

sys.path.append(r"C:\Program Files\Thorlabs\Kinesis")
os.getcwd()
clr.AddReference("Thorlabs.MotionControl.DeviceManagerCLI")
clr.AddReference("Thorlabs.MotionControl.FilterFlipperCLI")
from instrumental.drivers.motion.filter_flipper import Position

In [4]:
ff      = instrument("Thorlabs_FilterFlipper",reopen_policy="reuse")

KeyError: 16

In [ ]:
cam = instrument("Thorlabs_camera",reopen_policy="reuse")

ff = instrument(paramsets[2])
ff.save_instrument('Thorlabs_FilterFlipper')

daq = instrument(paramsets[1])
daq.save_instrument('NIDAQ_USB-6259')

daq = instrument('NIDAQ_USB-6259')

In [ ]:
#Configure DAQ input channels
Vi, Vi_str = daq.ai0, 'Dev2/ai0'
#configure DAQ output channels
Vo, Vo_str = daq.ao0, 'Dev2/ao0'

n_samp = 100
fsamp = 1000*u.Hz
test_task = Task(Vi, Vo)
test_task.set_timing(fsamp = fsamp, n_samples=n_samp)


In [ ]:
Vo.write(0*u.V)

In [ ]:
V_data = np.linspace(0,1,n_samp//2)*u.V
V_data = np.concatenate((V_data, V_data[::-1]))
# V_data = np.ones(5)*0*u.V
write_data = {Vo_str: V_data}
read_data = test_task.run(write_data)
test_task.unreserve()
print(read_data)
print(V_data)

# Testing plotting